In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
from torchvision import transforms

import pandas as pd
import math
import os
from skimage import io, transform
import numpy as np
import sys
#status bar
from tqdm import tqdm

#import exactly in this way to make sure that matplotlib can generate
#a plot without being connected to a display 
#(otherwise _tkinter.TclError: couldn't connect to display localhost:10.0)
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#Dataset and Transforms
from utils.Dataset_And_Transforms import FigrimFillersDataset, Downsampling, ToTensor, ExpandTargets, Targets2D
#Loss-Function
from utils.MyLoss import myLoss 
#Early stopping
from utils.EarlyStopping import EarlyStopping
#Adapted Sigmoid Function
from utils.MyActivationFunctions import mySigmoid
#Live-Plotting with Visdom
from utils.MyVisdom import VisdomLinePlotter
#evaluation
from utils.Evaluate_Baseline import map_idx, accuracy

In [18]:
a = torch.randn(128,128,100,100)
b = torch.ones(128,1,100,100)
c = torch.cat((a,b),1) #concatenate along channel dimension
c.size()

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
        28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
        42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55.,
        56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69.,
        70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82., 83.,
        84., 85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95., 96., 97.,
        98., 99.])

In [14]:
b = torch.ones(1,1,100,100)
c = b.repeat(128,1,1,1)
c.size()

torch.Size([128, 1, 100, 100])

In [17]:
torch.randn(5,5)

tensor([[ 1.3748,  0.5603, -1.4218, -0.4523, -1.3813],
        [ 1.1592, -1.4968, -0.2917, -0.4210,  0.6145],
        [-0.3284,  0.8345,  0.6765, -0.2781, -0.7563],
        [ 1.0577,  0.9099, -0.1743,  0.4969,  0.6981],
        [ 0.9722, -0.1175,  0.4325, -1.4201, -1.2895]])

In [4]:
a = torch.load('resnet-outputs.pt')[0]
a = a.view(a.size()[-3],a.size()[-2],a.size()[-1])
a.size()

torch.Size([512, 4, 4])

In [44]:
#Center Bias wie bei Schuett
a = torch.randn(100,100)
sigmax = 55
sigmay = 55
gridx = torch.Tensor(range(a.size()[-2]))
gridx = gridx - torch.mean(gridx)
gridx = gridx ** 2
gridx = gridx / (sigmax ** 2)
#so that gridx + gridy will give row- and column-wise combination
gridx = gridx.view(gridx.size()[0],1)

gridy = torch.Tensor(range(a.size()[-1]))
gridy = gridy - torch.mean(gridy)
gridy = gridy ** 2
gridy = gridy / (sigmay ** 2)

grid = gridx + gridy
CB = torch.exp(-0.5*grid)
CB = CB / torch.sum(CB)
CB


tensor([[5.7735e-05, 5.8678e-05, 5.9617e-05,  ..., 5.9617e-05, 5.8678e-05,
         5.7735e-05],
        [5.8678e-05, 5.9636e-05, 6.0590e-05,  ..., 6.0590e-05, 5.9636e-05,
         5.8678e-05],
        [5.9617e-05, 6.0590e-05, 6.1559e-05,  ..., 6.1559e-05, 6.0590e-05,
         5.9617e-05],
        ...,
        [5.9617e-05, 6.0590e-05, 6.1559e-05,  ..., 6.1559e-05, 6.0590e-05,
         5.9617e-05],
        [5.8678e-05, 5.9636e-05, 6.0590e-05,  ..., 6.0590e-05, 5.9636e-05,
         5.8678e-05],
        [5.7735e-05, 5.8678e-05, 5.9617e-05,  ..., 5.9617e-05, 5.8678e-05,
         5.7735e-05]])

In [73]:
def create_datasets(batch_size):
    
    #transforms
    data_transform = transforms.Compose([ToTensor(),Downsampling(10), Targets2D(100,100,100)])#ExpandTargets(100)])
    
    #load split data
    figrim_dataset_train = FigrimFillersDataset(json_file='allImages_unfolded_train.json',
                                        root_dir='figrim/fillerData/Fillers',
                                         transform=data_transform)

    figrim_dataset_val = FigrimFillersDataset(json_file='allImages_unfolded_val.json',
                                        root_dir='figrim/fillerData/Fillers',
                                         transform=data_transform)

    figrim_dataset_test = FigrimFillersDataset(json_file='allImages_unfolded_test.json',
                                        root_dir='figrim/fillerData/Fillers',
                                         transform=data_transform)
    
    #create data loaders
    #set number of threads to 8 so that 8 processes will transfer 1 batch to the gpu in parallel
    dataset_loader_train = torch.utils.data.DataLoader(figrim_dataset_train, batch_size=batch_size, 
                                             shuffle=True, num_workers=8)

    dataset_loader_val = torch.utils.data.DataLoader(figrim_dataset_val, batch_size=batch_size, 
                                                 shuffle=True, num_workers=8)
    
    
    #no shuffling, as to be able to identify which images were processed well/not so well
    dataset_loader_test = torch.utils.data.DataLoader(figrim_dataset_test, batch_size=batch_size, 
                                                 shuffle=False, num_workers=8)
    
    return dataset_loader_train, dataset_loader_val, dataset_loader_test

In [3]:
torch.Tensor([0.006, 0.061, 0.242, 0.383, 0.242, 0.061, 0.006]).repeat(5,1)

tensor([[0.0060, 0.0610, 0.2420, 0.3830, 0.2420, 0.0610, 0.0060],
        [0.0060, 0.0610, 0.2420, 0.3830, 0.2420, 0.0610, 0.0060],
        [0.0060, 0.0610, 0.2420, 0.3830, 0.2420, 0.0610, 0.0060],
        [0.0060, 0.0610, 0.2420, 0.3830, 0.2420, 0.0610, 0.0060],
        [0.0060, 0.0610, 0.2420, 0.3830, 0.2420, 0.0610, 0.0060]])

In [4]:
# Set these to whatever you want for your gaussian filter
kernel_size = 15
sigma = 3

# Create a x, y coordinate grid of shape (kernel_size, kernel_size, 2)
x_cord = torch.arange(kernel_size)
x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
y_grid = x_grid.t()
xy_grid = torch.stack([x_grid, y_grid], dim=-1)

mean = (kernel_size - 1)/2.
variance = sigma**2.

# Calculate the 2-dimensional gaussian kernel which is
# the product of two gaussian distributions for two different
# variables (in this case called x and y)
gaussian_kernel = (1./(2.*math.pi*variance)) *torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1) /(2*variance)
                  )
# Make sure sum of values in gaussian kernel equals 1.
gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)

# Reshape to 2d depthwise convolutional weight
gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
gaussian_kernel = gaussian_kernel.repeat(channels, 1, 1, 1)

gaussian_filter = nn.Conv2d(in_channels=channels, out_channels=channels,
                            kernel_size=kernel_size, groups=channels, bias=False)

gaussian_filter.weight.data = gaussian_kernel
gaussian_filter.weight.requires_grad = False

RuntimeError: "exp" not implemented for 'torch.LongTensor'

In [57]:
import numpy as np

def makeGaussian(size, fwhm = 3, center=None):
    """ Make a square gaussian kernel.
    size is the length of a side of the square
    fwhm is full-width-half-maximum, which
    can be thought of as an effective radius.
    """

    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]
    print(x)
    print(y)
    if center is None:
        x0 = y0 = size // 2
    else:
        x0 = center[0]
        y0 = center[1]
    
    return np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)

In [58]:
np.sum(makeGaussian(5, fwhm=3))

[0. 1. 2. 3. 4.]
[[0.]
 [1.]
 [2.]
 [3.]
 [4.]]


9.320802967661667

In [12]:
import math
import torch

#always with mean 0,0?
def gaussian(x,y,sigma):
    return 1/(2*math.pi*sigma**2) * math.exp(-(x**2+y**2)/2*sigma**2)

def gaussian_map(activations, sigma, gpu):
    """
    Takes:
    - activations: activations array (last two dimensions are used, these have to be odd and square)
    - sigma: standard deviation for 2D-Gauss-Function (mean is always (0,0)) [Tensor with one scalar]
    - gpu: bool: Do computations on gpu or cpu 
    Gives:
    - Array of the size of the last two dimensions with the 2D-Gauss-Function evaluated at each array entry
    """
    
    rows = activations.size()[-2]
    cols = activations.size()[-1]
    gaussian_map = torch.zeros(rows, cols)
    
    if gpu:
        if torch.cuda.is_available():
            gaussian_map = gaussian_map.to('cuda')
    
    mean_row = rows // 2
    mean_col = cols // 2

    for i in range(rows):
        for j in range(cols):
            gaussian_map[i,j] = gaussian(abs(i-mean_row),abs(j-mean_col),sigma.item())
    return gaussian_map

#square, uneven activations array
activations = torch.randn(1,1,15,15)
sigma = torch.Tensor([1])
gaussian_map(activations, sigma, True)

RuntimeError: value cannot be converted to type float without overflow: 1.43621e+44

In [5]:
class TestNet(nn.Module):

    def __init__(self, gpu=False):
        super(TestNet, self).__init__()
        #3 input image channels (color-images), 64 output channels,  3x3 square convolution kernel
        #padding to keep dimensions of output at 100x100
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.conv1_bn = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 1, 3, stride=1, padding=1)
        self.conv3_bn = nn.BatchNorm2d(1)
        #self.pool1 = nn.AdaptiveMaxPool2d((50,50))
        self.conv4 = nn.Conv2d(1, 1, 1, stride=1, padding=0)
        #pooling so that dimension is reduced by one (from 100 to 99), so that we have a middle pixel
        #for the Gaussian mask; stride=1 so that we only reduce by 1 pixel
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.gauss_sigma = nn.Parameter(torch.Tensor([1]))
        self.gauss_sigma.requires_grad_()
        #self.gauss.weight = gaussian_weights
        #scale parameter for the sigmoid function
        self.upper_bound = nn.Parameter(torch.Tensor([1]))
        #make it considered by autograd
        self.upper_bound.requires_grad_()
        self.gpu = gpu
        if gpu:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                self.cuda()
        
    def forward(self, x):
        #print("input sum at beginning of forward pass: {}".format(torch.sum(x)))
        x = functional.relu(self.conv1(x))
        #print("input sum after first conv and relu: {}".format(torch.sum(x)))
        x = self.conv1_bn(x)
        #print("input sum after first batch normalization: {}".format(torch.sum(x)))
        x = functional.relu(self.conv2(x))
        #print("input sum after second conv and relu: {}".format(torch.sum(x)))
        x = self.conv2_bn(x)
        #print("output shape: {}".format(x.size()))
        #print("input sum after second batch normalization: {}".format(torch.sum(x)))
        #if scaled by a negative value, we would try to take the ln of negative values in the loss  function
        #(ln is not defined for negative values), so make sure that the scaling parameter is positive
        #x = mySigmoid(self.conv3(x), abs(self.upper_bound), gpu)
        #x = functional.relu(self.conv3(x))
        x = functional.relu(self.conv3(x))
        x = self.conv3_bn(x)
        #print("input sum after last conv and sigmoid: {}".format(torch.sum(x)))
        #x = self.pool1(x)
        x = self.conv4(x)
        x = self.pool1(x)
        x = gaussian_map(x, self.gauss_sigma, gpu) * x
        
        return x

In [83]:
gpu = True

#initilaize the NN
model = TestNet(gpu)
train_loader, val_loader, test_loader = create_datasets(128)


In [88]:
#model = model.to('cpu')
t = iter(test_loader)
for i, example in enumerate(t): #start at index 0
            # get the inputs
            data = example["image"]
            #print("input sum: {}".format(torch.sum(data)))
            target = example["fixations"]
            #target_locs = example["fixation_locs"]
            
            data = data.to('cuda')
            target = target.to('cuda')
            
            output = model(data)
            if i == 0:
                break
print(output[0,0,50,50])

tensor(0.0179, device='cuda:0', grad_fn=<SelectBackward>)


In [71]:
gaussian(abs(0-1),abs(0-1),1)


0.05854983152431917

In [32]:
math.exp(-(1**2+1**2)/2*18**2)

1.9435148500492928e-141

In [33]:
1/(2*math.pi*18**2)

0.0004912189601601709

In [25]:
x = np.arange(0,5,1,float)
y = x[:, np.newaxis]
y
torch.from_numpy(makeGaussian(5)).size()

torch.Size([5, 5])

In [ ]:
0.003765	0.015019	0.023792	0.015019	0.003765
0.015019	0.059912	0.094907	0.059912	0.015019
0.023792	0.094907	0.150342	0.094907	0.023792
0.015019	0.059912	0.094907	0.059912	0.015019
0.003765	0.015019	0.023792	0.015019	0.003765

In [4]:
class TestNet(nn.Module):

    def __init__(self, gpu=False):
        super(TestNet, self).__init__()
        #3 input image channels (color-images), 64 output channels,  3x3 square convolution kernel
        #padding to keep dimensions of output at 100x100
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        print(self.conv1.weight.size())
        self.conv1_bn = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 1, 3, stride=1, padding=1)
        self.conv3_bn = nn.BatchNorm2d(1)
        #self.pool1 = nn.AdaptiveMaxPool2d((50,50))
        self.conv4 = nn.Conv2d(1, 1, 1, stride=1, padding=0)
        self.gauss = nn.Conv2d(1, 1, 5, padding =2, bias=False)
        #has to have size (1,1,5,5), as 1 output channel, 1 input channel, 5x5 convolution
        self.gauss.weight = nn.Parameter(torch.from_numpy(makeGaussian(5)).view(1,1,5,5).float(), requires_grad=False)
        print(self.gauss.weight)
        #self.gauss.weight = gaussian_weights
        #scale parameter for the sigmoid function
        self.upper_bound = nn.Parameter(torch.Tensor([1]))
        #make it considered by autograd
        self.upper_bound.requires_grad_()
        self.gpu = gpu
        if gpu:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                self.cuda()
        
    def forward(self, x):
        #print("input sum at beginning of forward pass: {}".format(torch.sum(x)))
        x = functional.relu(self.conv1(x))
        #print("input sum after first conv and relu: {}".format(torch.sum(x)))
        x = self.conv1_bn(x)
        #print("input sum after first batch normalization: {}".format(torch.sum(x)))
        x = functional.relu(self.conv2(x))
        #print("input sum after second conv and relu: {}".format(torch.sum(x)))
        x = self.conv2_bn(x)
        #print("output shape: {}".format(x.size()))
        #print("input sum after second batch normalization: {}".format(torch.sum(x)))
        #if scaled by a negative value, we would try to take the ln of negative values in the loss  function
        #(ln is not defined for negative values), so make sure that the scaling parameter is positive
        #x = mySigmoid(self.conv3(x), abs(self.upper_bound), gpu)
        #x = functional.relu(self.conv3(x))
        x = functional.relu(self.conv3(x))
        x = self.conv3_bn(x)
        #print("input sum after last conv and sigmoid: {}".format(torch.sum(x)))
        #x = self.pool1(x)
        x = self.conv4(x)
        x = self.gauss(x)
        
        return x

#initilaize the NN
model = TestNet(True)

torch.Size([64, 3, 3, 3])
Parameter containing:
tensor([[[[0.0850, 0.2143, 0.2916, 0.2143, 0.0850],
          [0.2143, 0.5400, 0.7349, 0.5400, 0.2143],
          [0.2916, 0.7349, 1.0000, 0.7349, 0.2916],
          [0.2143, 0.5400, 0.7349, 0.5400, 0.2143],
          [0.0850, 0.2143, 0.2916, 0.2143, 0.0850]]]])


In [29]:
#model = model.to('cpu')
t = iter(test_loader)
for i, example in enumerate(t): #start at index 0
            # get the inputs
            data = example["image"]
            #print("input sum: {}".format(torch.sum(data)))
            target = example["fixations"]
            #target_locs = example["fixation_locs"]
            
            data = data.to('cuda')
            target = target.to('cuda')
            
            output = model(data)
            if i == 0:
                break
print(output[0])

tensor([[[ -6.0951,  -8.7581, -10.0444,  ..., -10.4240,  -9.5387,  -6.9138],
         [ -8.5760, -12.3445, -14.0827,  ..., -14.6083, -13.2694,  -9.5698],
         [ -9.7867, -14.0851, -15.9635,  ..., -16.5652, -14.8720, -10.6190],
         ...,
         [ -4.6591,  -6.3251,  -6.9735,  ...,  -6.9735,  -6.3251,  -4.6592],
         [ -4.2749,  -5.7656,  -6.3251,  ...,  -6.3124,  -5.7336,  -4.2314],
         [ -3.1967,  -4.2749,  -4.6591,  ...,  -4.6325,  -4.2078,  -3.1053]]],
       device='cuda:0', grad_fn=<SelectBackward>)


In [26]:
criterion = nn.PoissonNLLLoss(log_input=True, full=True, reduction="mean")

In [30]:
output = output.view(-1, target.size()[-1], target.size()[-2])
loss = criterion(output, target)
loss

tensor(0.0068, device='cuda:0', grad_fn=<MeanBackward1>)

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.25)

In [28]:
loss.backward()
optimizer.step()

In [18]:
from utils.MyLoss import myLoss2

In [21]:
loss = myLoss2(output, target)
loss.backward()

In [14]:
print(output)

tensor([[[[-1.5839e+01, -2.0886e+01, -1.2744e+01,  ..., -1.7392e+01,
           -1.5468e+01,  1.3891e+00],
          [-1.5319e+01, -1.2040e+01, -8.3850e+00,  ..., -6.8545e+00,
           -1.3909e+01,  3.7658e+00],
          [-1.9382e+01, -1.4894e+01, -6.8167e+00,  ..., -5.5862e+00,
           -1.6090e+01,  5.4470e+00],
          ...,
          [-9.4274e+00, -5.4895e+00, -2.4079e-01,  ...,  1.9000e+00,
           -6.5631e+00,  6.4394e+00],
          [-9.0873e+00, -3.3145e+00,  1.4876e+00,  ...,  4.3158e+00,
           -2.4395e+00,  6.7142e+00],
          [-3.6674e+00,  2.6020e+00,  2.6126e+00,  ...,  8.2902e+00,
            7.0707e-01,  5.9727e+00]]],


        [[[-1.1118e+01, -1.2144e+01, -8.8363e+00,  ..., -1.0303e+01,
           -1.0382e+01, -3.4797e-01],
          [-1.2356e+01, -7.9673e+00, -7.1867e+00,  ..., -5.7309e+00,
           -8.2004e+00, -1.7581e-01],
          [-1.3366e+01, -1.1009e+01, -6.9016e+00,  ..., -5.9062e+00,
           -8.2549e+00,  2.1992e+00],
          ...,
   

In [309]:
output = torch.randn(1,1,100,100)
fixations1 = torch.Tensor(([44,44]))
fixations1 = fixations1.view(1,1,2).long()
fixations2 = torch.zeros(1,100,100)
fixations2[0,44,44] = 1
#fixations2[0,23,32] = 1
fixations1.size()

torch.Size([1, 1, 2])

In [311]:
torch.sum(output) - torch.log(output[0,0,44,44])

tensor(-81.2100)

In [312]:
loss1 = myLoss(output, fixations1)
loss1

tensor(0.)

In [330]:
criterion = nn.PoissonNLLLoss(log_input=False)
loss2 = criterion(output.view(1,100,100), fixations2)
loss2

tensor(nan)

In [331]:
torch.sum(output.view(1,100,100) - torch.log(output.view(1,100,100))*fixations2)

tensor(nan)

In [326]:
(output.view(1,100,100)*fixations2).size()

torch.Size([1, 100, 100])

In [263]:
loss1

tensor(639734.8750, grad_fn=<SumBackward0>)

In [257]:
loss2

tensor(0.4964, device='cuda:0', grad_fn=<MeanBackward1>)

In [222]:
output.size()

torch.Size([128, 1, 100, 100])

In [203]:
def map_idx(tensor_unfl, idx_fl):
    """
    Takes unflattened 2D-tensor and index of the same flattened 2D-tensor and returns the corresponding index
    of the unflattened tensor.
    """
    #row_number of unflattened tensor is index of flattened tensor // amount of columns of unflattened tensor
    #col_number of unflattened tensor is index of flattened tensor % amount of columns of unflattened tensor
    n_cols = tensor_unfl.size()[-1]
    row_idx_unfl = idx_fl // n_cols
    col_idx_unfl = idx_fl % n_cols
    #result = torch.tensor([row_idx_unfl, col_idx_unfl])
    #if gpu:
    #    if torch.cuda.is_available():
    #        result = result.to('cuda')
    return (row_idx_unfl, col_idx_unfl)


# In[3]:


def accuracy(activations, fixations, fixation_locs, gpu):
    """
    Calculates the accuracy for one image's activations and its corresponding fixation sequence.
    
    Takes: - activations: tensor of activations of size (1,act_x,act_y) (1 channel)
           - fixations: tensor of fixations sequence of size (x,2)
    
    Returns: - accuracy for this image: 1, if eg 4 fixations, and each fixation location is equal to one of the 
               the 4 biggest activation values. So eg 0.75, if only 3 fixation locations hit one of the four biggest 
               activation values and so on
            - hits for this image: how many fixations led to one of the x biggest activation values?
            - number of fixations this image had in this trial
    """

    ##Accuracy for one image
    
    #already done in the training/val/eval loop
    #drop unnecessary first dimension of activations (there is only one channel)
    #activations = activations.reshape(activations.size()[-2], activations.size()[-1])

    #how many fixations are there?
    #num_fix = 0
    #for i,j in fixations:
    #            if (i,j) == (-1000,-1000):
    #                break
    #            num_fix += 1
                
    #extract the fixated locations
    #locations = []
    #for i in range(fixations.size()[0]):
    #    for j in range(fixations.size()[1]):
    #        if fixations[i,j] != 0:
    #            locations.append((i,j))
    
    #extract fixated locations from tensor form
    #locations = []
    #for i in range(len(fixation_locs)):
    #    locations.append(tuple(fixation_locs.tolist()[i]))
                
    #How many fixations are there?
    num_fix = int(torch.sum(fixations).item())
    
    #select only the first num_fix entries of fixation_locs (rest is (-1000,-1000))
    fixation_locs = fixation_locs[0:num_fix]
    
    #extract fixated locations from expanded tensor
    locations = []
    fixations_l = fixation_locs.tolist()
    #everything coming after the indx num_fix is only (-1000,-1000)
    for i in range(num_fix):
        locations.append(tuple(fixations_l[i]))
    
    #flatten activations
    activations_f = activations.view(-1)

    #find x largest values and their indices in flattened activation-tensor
    lar_val, lar_val_idx = torch.topk(activations_f, num_fix)
    
    idx_unfl = []
    for idx_fl in lar_val_idx:
        idx_unfl.append(map_idx(activations, idx_fl.item()))

    #see if they match with fixations indices
    #hits = 0
    #does each fixation lead to one of the x biggest activation values?
    #for fix in range(num_fix):
    #    for idx in idx_unfl:
    #        current = torch.all(torch.eq(idx,fixations[fix]))
    #        hits += current.item()
    
    #see if they match with fixations indices
    hits = 0
    
    #print("Fixation locations: {}".format(locations))
    #print("Indices of biggest activations: {}".format(idx_unfl))
    
    #does each fixation lead to one of the x biggest activation values?
    biggest_activation_locs = torch.zeros(activations.size())
    
    for idx in idx_unfl:
        biggest_activation_locs[idx] = 1
    
    if gpu:
        if torch.cuda.is_available():
            biggest_activation_locs = biggest_activation_locs.to('cuda')
    
    hits = int(torch.sum(biggest_activation_locs * fixations).item())
    
    #for fix in locations:
    #    for idx in idx_unfl:
            #convert the tensor holding the fixation value to 
            #current = torch.all(torch.eq(idx, fixations[fix].long()))
            #hits += current.item()
    #        if fix == idx:
    #            hits += 1
    
    #calcualte proportion of hits
    acc = hits / num_fix
    
    return acc, hits, num_fix


In [204]:
model.load_state_dict(torch.load("checkpoint_batch_size_128_lr_0.0001.pt"))

In [206]:
criterion = nn.PoissonNLLLoss(log_input=False)
gpu = True

#evaluate the model
# to track the training loss as the model trains
test_losses = []
#to track the accuracy 
acc_per_image = []
acc_per_batch = []
#track absolute hits
hit_list = []
#track number of fixations
n_fixations = []

model.eval() # prep model for evaluation
t = iter(train_loader)
for i, example in enumerate(t): #start at index 0
            # get the inputs
            data = example["image"]
            #print("input sum: {}".format(torch.sum(data)))
            target = example["fixations"]
            target_locs = example["fixation_locs"]
            
            #push data and targets to gpu
            if gpu:
                if torch.cuda.is_available():
                    data = data.to('cuda')
                    target = target.to('cuda')
            
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            
            #drop channel-dimension (is only 1) so that outputs will be of same size as targets (batch_size,100,100)
            output = output.view(-1, target.size()[-2], target.size()[-2])
            
            loss = criterion(output, target)
            # calculate the loss
            #loss = myLoss(output, target)
            # record training loss
            test_losses.append(loss.item())
            #accuracy
            acc_this_batch = 0
            for batch_idx in range(output.size()[0]):
                output_subset = output[batch_idx]
                target_subset = target[batch_idx]
                target_locs_subset = target_locs[batch_idx]
                acc_this_image, hits, num_fix = accuracy(output_subset, target_subset, target_locs_subset, gpu)
                acc_per_image.append(acc_this_image)
                hit_list.append(hits)
                n_fixations.append(num_fix)
                acc_this_batch += acc_this_image
            #divide by batch size
            acc_this_batch /= output.size()[0]
            acc_per_batch.append(acc_this_batch)
                
acc_per_image = np.asarray(acc_per_image)
print("Mean test loss is: {}".format(np.average(test_losses)))
print("Mean accuracy for test set ist: {}".format(np.mean(acc_per_image)))
print("Hits: {}".format(sum(hit_list)))

Mean test loss is: 0.01585181766652806
Mean accuracy for test set ist: 0.000543421101674545
Hits: 95


In [163]:
import torch
import numpy as np


# In[2]:


#find indices of these values in unflattend activation-tensor
def map_idx(tensor_unfl, idx_fl):
    """
    Takes unflattened 2D-tensor and index of the same flattened 2D-tensor and returns the corresponding index
    of the unflattened tensor.
    """
    #row_number of unflattened tensor is index of flattened tensor // amount of columns of unflattened tensor
    #col_number of unflattened tensor is index of flattened tensor % amount of columns of unflattened tensor
    n_cols = tensor_unfl.size()[-1]
    row_idx_unfl = idx_fl // n_cols
    col_idx_unfl = idx_fl % n_cols
    #result = torch.tensor([row_idx_unfl, col_idx_unfl])
    #if gpu:
    #    if torch.cuda.is_available():
    #        result = result.to('cuda')
    return (row_idx_unfl, col_idx_unfl)


# In[3]:


def accuracy(activations, fixations, fixation_locs):
    """
    Calculates the accuracy for one image's activations and its corresponding fixation sequence.
    
    Takes: - activations: tensor of activations of size (1,act_x,act_y) (1 channel)
           - fixations: tensor of fixations sequence of size (x,2)
    
    Returns: - accuracy for this image: 1, if eg 4 fixations, and each fixation location is equal to one of the 
               the 4 biggest activation values. So eg 0.75, if only 3 fixation locations hit one of the four biggest 
               activation values and so on
            - hits for this image: how many fixations led to one of the x biggest activation values?
            - number of fixations this image had in this trial
    """

    ##Accuracy for one image
    
    #already done in the training/val/eval loop
    #drop unnecessary first dimension of activations (there is only one channel)
    #activations = activations.reshape(activations.size()[-2], activations.size()[-1])

    #how many fixations are there?
    #num_fix = 0
    #for i,j in fixations:
    #            if (i,j) == (-1000,-1000):
    #                break
    #            num_fix += 1
                
    #extract the fixated locations
    #locations = []
    #for i in range(fixations.size()[0]):
    #    for j in range(fixations.size()[1]):
    #        if fixations[i,j] != 0:
    #            locations.append((i,j))
    
    #extract fixated locations from tensor form
    #locations = []
    #for i in range(len(fixation_locs)):
    #    locations.append(tuple(fixation_locs.tolist()[i]))
                
    #How many fixations are there?
    num_fix = int(torch.sum(fixations).item())
    
    #extract fixated locations from expanded tensor
    locations = []
    fixations_l = fixation_locs.tolist()
    #everything coming after the indx num_fix is only (-1000,-1000)
    for i in range(num_fix):
        locations.append(tuple(fixations_l[i]))
    
    #flatten activations
    activations_f = activations.view(-1)

    #find x largest values and their indices in flattened activation-tensor
    lar_val, lar_val_idx = torch.topk(activations_f, num_fix)
    
    idx_unfl = []
    for idx_fl in lar_val_idx:
        idx_unfl.append(map_idx(activations, idx_fl.item()))

    #see if they match with fixations indices
    #hits = 0
    #does each fixation lead to one of the x biggest activation values?
    #for fix in range(num_fix):
    #    for idx in idx_unfl:
    #        current = torch.all(torch.eq(idx,fixations[fix]))
    #        hits += current.item()
    
    #see if they match with fixations indices
    hits = 0
    #does each fixation lead to one of the x biggest activation values?
    for fix in locations:
        for idx in idx_unfl:
            #convert the tensor holding the fixation value to 
            #current = torch.all(torch.eq(idx, fixations[fix].long()))
            #hits += current.item()
            if fix == idx:
                hits += 1
    
    #calcualte proportion of hits
    acc = hits / num_fix
    
    return acc, hits, num_fix



In [165]:
for i, example in enumerate(test_loader, 0):
    if i == 0:
        output = torch.randn(100,100)
        fixations = example["fixations"]
        fixation_locs = example["fixation_locs"]
        print(fixation_locs[0].size())
        acc, hits, num_fix = accuracy(output, fixations[0], fixation_locs[0])
        #print()

torch.Size([100, 2])


In [176]:
fixation_locs
output[55,45] = 1000000000

In [177]:
acc, hits, num_fix = accuracy(output, fixations[0], fixation_locs[0])

In [178]:
hits

1

In [179]:
acc

0.1111111111111111

In [180]:
1/9

0.1111111111111111

In [187]:
test = fixation_locs[0]
test[0:5]

tensor([[55, 45],
        [46, 52],
        [36, 67],
        [31, 72],
        [63, 70]])

In [186]:
fixation_locs.size()

torch.Size([128, 100, 2])

In [ ]:
lambda1 = lambda epoch: epoch // 30

In [1]:
import torch

In [3]:
a = torch.Tensor(([1,2,3],[4,5,6]))
a

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [4]:
b = a.view(-1)
b

tensor([1., 2., 3., 4., 5., 6.])

In [7]:
for elem in b:
    print(elem.item())

1.0
2.0
3.0
4.0
5.0
6.0


In [2]:
import torch
a = torch.randn(128,1,100,100)
b = torch.randn(100,100)
(a * b).size()

torch.Size([128, 1, 100, 100])

In [6]:
a = torch.Tensor([2])
a ** 2

tensor([4.])

In [7]:
abs(a-5)

tensor([3.])